# --- Parte 1: web_scrapping_bumeran.ipynb ---

In [25]:
## EJERCICIO 2 
## Bumeran — Web Scraping
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install requests beautifulsoup4 pandas lxml
!{sys.executable} -m pip install selenium webdriver-manager


In [26]:
## Imports y helpers
import os, re, time, json, datetime as dt
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Carpeta de salida (tú usas "output")
OUT_DIR = "../output"
os.makedirs(OUT_DIR, exist_ok=True)

UAS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome Safari",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:117.0) Gecko/20100101 Firefox/117.0",
]
def ua(i=0): return {"User-Agent": UAS[i % len(UAS)]}

def make_session():
    s = requests.Session()
    r = Retry(total=3, backoff_factor=0.6, status_forcelist=[429,500,502,503,504])
    s.mount("http://", HTTPAdapter(max_retries=r))
    s.mount("https://", HTTPAdapter(max_retries=r))
    return s

def log(msg): 
    print(f"[{dt.datetime.now().strftime('%H:%M:%S')}] {msg}")


In [27]:
## Usamos la funcion get_listing_urls_requests
def get_listing_urls_requests(base_url_pattern: str, max_pages: int = 3, pause: float = 1.0):
    sess = make_session()
    urls = []
    for page in range(1, max_pages+1):
        url = base_url_pattern.format(page=page) if "{page}" in base_url_pattern \
              else (base_url_pattern if page == 1 else re.sub(r"(\.html)?$", f"-pagina-{page}.html", base_url_pattern))
        resp = sess.get(url, headers=ua(page-1), timeout=25)
        log(f"GET {resp.status_code} {url} (len={len(resp.text)})")
        if resp.status_code != 200 or len(resp.text) < 1000:
            if page == 1: break
            else: continue
        soup = BeautifulSoup(resp.text, "lxml")

        # Tarjetas (probamos varios selectores)
        cards = soup.select("[data-test-id='job-item']") or soup.select(".job-card") or soup.select("article")

        # además, plan B: escanear todos los <a>
        anchors = soup.select("a[href]")
        found = 0
        for a in anchors:
            href = a.get("href") or ""
            full = href if href.startswith("http") else urljoin("https://www.bumeran.com.pe", href)
            if urlparse(full).netloc.endswith("bumeran.com.pe") and re.search(r"(oferta-de-trabajo|empleo|empleos)", full):
                urls.append(full); found += 1
        log(f"Enlaces válidos encontrados: {found} · Acumulado: {len(urls)}")
        time.sleep(pause)
    return list(dict.fromkeys(urls))


In [28]:
## Listado con selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def get_listing_urls_selenium(base_url_pattern: str, max_pages: int = 2, pause: float = 1.2, headless=False):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--start-maximized")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=opts)

    urls = []
    try:
        for page in range(1, max_pages+1):
            url = base_url_pattern.format(page=page) if "{page}" in base_url_pattern \
                  else (base_url_pattern if page == 1 else re.sub(r"(\.html)?$", f"-pagina-{page}.html", base_url_pattern))
            log(f"Selenium GET {url}")
            driver.get(url)

            # Intentar cerrar cookie banner
            clicked = False
            for sel in [
                "button#onetrust-accept-btn-handler",
                "button[aria-label*='Aceptar']",
                "button[aria-label*='Aceptar todo']",
                "button[aria-label*='Aceptar todas']",
            ]:
                try:
                    el = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, sel)))
                    el.click(); clicked = True; break
                except Exception:
                    pass
            if not clicked:
                for txt in ["Aceptar", "Acepto", "Aceptar todo", "Aceptar y cerrar", "Entendido"]:
                    try:
                        btn = driver.find_element(By.XPATH, f"//button[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZÁÉÍÓÚÜ', 'abcdefghijklmnopqrstuvwxyzáéíóúü'), '{txt.lower()}')]")
                        btn.click(); break
                    except Exception:
                        pass

            # esperar algo de contenido y scrollear para lazy-load
            try:
                WebDriverWait(driver, 8).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "[data-test-id='job-item'], .job-card, article"))
                )
            except Exception:
                log("No se detectaron tarjetas enseguida. Sigo igual.")
            prev_h = 0
            for _ in range(8):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1.0)
                h = driver.execute_script("return document.body.scrollHeight")
                if h == prev_h: break
                prev_h = h

            soup = BeautifulSoup(driver.page_source, "lxml")
            found = 0
            for a in soup.select("a[href]"):
                href = a.get("href") or ""
                full = href if href.startswith("http") else urljoin("https://www.bumeran.com.pe", href)
                if "bumeran.com.pe" in full and re.search(r"(oferta-de-trabajo|empleo|empleos)", full):
                    urls.append(full); found += 1
            log(f"[Sel] Enlaces válidos: {found} · Acum: {len(urls)}")
            time.sleep(pause)
    finally:
        driver.quit()

    return list(dict.fromkeys(urls))


In [29]:
## Orquestador del listado
def get_job_urls_auto(base_url, max_pages=2, pause=1.2):
    urls = get_listing_urls_requests(base_url, max_pages=max_pages, pause=pause)
    if not urls:
        log("➡️ Pasando a Selenium (ventana visible) …")
        urls = get_listing_urls_selenium(base_url, max_pages=max_pages, pause=pause, headless=False)
    log(f"Total URLs capturadas: {len(urls)}")
    return urls


In [30]:
## definimos la url
BASE_URL = "https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html"

MAX_PAGES = 2     # sube a 3–4 si ves más resultados (sin abusar)
PAUSE = 1.2

job_urls = get_job_urls_auto(BASE_URL, max_pages=MAX_PAGES, pause=PAUSE)
log(f"Primeras URLs: {job_urls[:5]}")


[19:31:30] GET 200 https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html (len=63597)
[19:31:30] Enlaces válidos encontrados: 0 · Acumulado: 0
[19:31:31] GET 200 https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias-pagina-2.html-pagina-2.html (len=63597)
[19:31:31] Enlaces válidos encontrados: 0 · Acumulado: 0
[19:31:32] ➡️ Pasando a Selenium (ventana visible) …
[19:31:35] Selenium GET https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html
[19:31:58] No se detectaron tarjetas enseguida. Sigo igual.
[19:32:00] [Sel] Enlaces válidos: 60 · Acum: 60
[19:32:01] Selenium GET https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-m

In [31]:
## extraemos la informacion
def parse_jsonld_job(soup: BeautifulSoup):
    data = {"title": None, "description": None, "district": None, "employmentType": None, "company": None}
    for sc in soup.find_all("script", type="application/ld+json"):
        try: obj = json.loads(sc.string or "")
        except Exception: continue
        items = obj if isinstance(obj, list) else [obj]
        for it in items:
            if not isinstance(it, dict): continue
            t = it.get("@type")
            if isinstance(t, list): t = t[0]
            if t == "JobPosting" or any(k in it for k in ["title","description","hiringOrganization"]):
                data["title"] = data["title"] or it.get("title") or it.get("name")
                data["description"] = data["description"] or it.get("description")
                emp = it.get("employmentType"); 
                if isinstance(emp, list): emp = ", ".join(emp)
                data["employmentType"] = data["employmentType"] or emp
                org = it.get("hiringOrganization")
                if isinstance(org, dict): data["company"] = data["company"] or org.get("name")
                loc = it.get("jobLocation")
                if isinstance(loc, list) and loc: loc = loc[0]
                if isinstance(loc, dict):
                    addr = loc.get("address")
                    if isinstance(addr, dict): data["district"] = data["district"] or addr.get("addressLocality")
    return data

def extract_job_detail_requests(job_url: str) -> dict:
    sess = make_session()
    r = sess.get(job_url, headers=ua(0), timeout=25)
    if r.status_code != 200:
        return {"url": job_url, "Titulo del puesto": None, "Descripcion": None, "Distrito": None, "Modo de trabajo": None, "empresa": None}
    soup = BeautifulSoup(r.text, "lxml")
    jld = parse_jsonld_job(soup)
    title = jld.get("title")
    desc  = jld.get("description")
    company = jld.get("company")
    district = jld.get("district")
    mode = jld.get("employmentType")

    if not title:
        t = soup.select_one("[data-test-id='job-title']") or soup.find("h1")
        title = t.get_text(strip=True) if t else None
    if not company:
        c = soup.select_one("[data-test-id='job-company']") or soup.find(class_=re.compile("company", re.I))
        company = c.get_text(strip=True) if c else None
    if not desc:
        dc = soup.find(id=re.compile("descripcion|description", re.I)) or soup.find("section", string=re.compile("Descripción", re.I)) or soup.find("section")
        if dc:
            raw = dc.get_text("\n", strip=True)
            parts = re.split(r"(?i)beneficios", raw); desc = parts[0].strip() if parts else raw
    if not district:
        chip = soup.find(string=re.compile("Lima", re.I))
        if chip and getattr(chip, "parent", None): district = chip.parent.get_text(strip=True)
    if not mode:
        for key in ["remoto","híbrido","hibrido","presencial","full-time","tiempo completo"]:
            m = soup.find(string=re.compile(key, re.I))
            if m: mode = m.strip(); break

    return {"url": job_url, "Titulo del puesto": title, "Descripcion": desc,
            "Distrito": district, "Modo de trabajo": mode, "empresa": company}

# Fallback Selenium para detalle (por si alguna página también es JS-heavy)
from selenium.webdriver.chrome.service import Service
def extract_job_detail_selenium(job_url: str) -> dict:
    opts = Options(); opts.add_argument("--no-sandbox"); opts.add_argument("--disable-gpu"); opts.add_argument("--headless=new")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=opts)
    try:
        driver.get(job_url)
        WebDriverWait(driver, 8).until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1, [data-test-id='job-title']")))
        soup = BeautifulSoup(driver.page_source, "lxml")
        jld = parse_jsonld_job(soup)
        title = jld.get("title") or (soup.select_one("[data-test-id='job-title']") or soup.find("h1"))
        title = title if isinstance(title, str) else (title.get_text(strip=True) if title else None)
        company = jld.get("company") or (soup.select_one("[data-test-id='job-company']") or soup.find(class_=re.compile("company", re.I)))
        company = company if isinstance(company, str) else (company.get_text(strip=True) if company else None)
        desc = jld.get("description")
        if not desc:
            dc = soup.find(id=re.compile("descripcion|description", re.I)) or soup.find("section", string=re.compile("Descripción", re.I)) or soup.find("section")
            if dc:
                raw = dc.get_text("\n", strip=True)
                parts = re.split(r"(?i)beneficios", raw); desc = parts[0].strip() if parts else raw
        district = jld.get("district")
        if not district:
            chip = soup.find(string=re.compile("Lima", re.I))
            if chip and getattr(chip, "parent", None): district = chip.parent.get_text(strip=True)
        mode = jld.get("employmentType")
        if not mode:
            for key in ["remoto","híbrido","hibrido","presencial","full-time","tiempo completo"]:
                m = soup.find(string=re.compile(key, re.I))
                if m: mode = m.strip(); break
        return {"url": job_url, "Titulo del puesto": title, "Descripcion": desc,
                "Distrito": district, "Modo de trabajo": mode, "empresa": company}
    finally:
        driver.quit()

def extract_job_detail_auto(job_url: str) -> dict:
    d = extract_job_detail_requests(job_url)
    if not any([d["Titulo del puesto"], d["Descripcion"], d["Distrito"], d["Modo de trabajo"]]):
        return extract_job_detail_selenium(job_url)
    return d


In [32]:
rows = []
for i, u in enumerate(job_urls, start=1):
    log(f"[{i}/{len(job_urls)}] Detalle -> {u}")
    rows.append(extract_job_detail_auto(u))
    time.sleep(0.6)

expected = ["url","Titulo del puesto","Descripcion","Distrito","Modo de trabajo","empresa"]
df_jobs = pd.DataFrame(rows, columns=expected)
log(f"Filas extraídas: {len(df_jobs)}")

if not df_jobs.empty:
    df_jobs["Titulo del puesto"] = df_jobs["Titulo del puesto"].astype("string").str.strip()
    df_jobs["Descripcion"] = df_jobs["Descripcion"].astype("string").str.replace(r"\s+\n","\n", regex=True).str.strip()
    display(df_jobs.head())
else:
    log("⛔ df_jobs está vacío. Revisa que BASE_URL tenga resultados y, si es necesario, sube MAX_PAGES.")


[19:32:29] [1/61] Detalle -> https://www.bumeran.com.pe/empleos-seniority-junior.html?landing-jovenes-profesionales=true
[19:32:35] [2/61] Detalle -> https://www.bumeran.com.pe/empleos-seniority-gerencia-alta-gerencia-direccion.html?landing-puestos-ejecutivos=true
[19:32:42] [3/61] Detalle -> https://www.bumeran.com.pe/empleos.html
[19:32:49] [4/61] Detalle -> https://www.bumeran.com.pe/empleos-area-tecnologia-sistemas-y-telecomunicaciones.html
[19:32:55] [5/61] Detalle -> https://www.bumeran.com.pe/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion.html
[19:33:01] [6/61] Detalle -> https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion.html
[19:33:08] [7/61] Detalle -> https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html?relevantes=true
[19:33:14] [8/61] Detalle -> https://www.bumeran.com.pe/en-lima/empleos-a

,url,Titulo del puesto,Descripcion,Distrito,Modo de trabajo,empresa
0,https://www.bumeran.com.pe/empleos-seniority-j...,"Sorry, you have been blocked",<NA>,None,None,None
1,https://www.bumeran.com.pe/empleos-seniority-g...,"Sorry, you have been blocked",<NA>,None,None,None
2,https://www.bumeran.com.pe/empleos.html,"Sorry, you have been blocked",<NA>,None,None,None
3,https://www.bumeran.com.pe/empleos-area-tecnol...,"Sorry, you have been blocked",<NA>,None,None,None
4,https://www.bumeran.com.pe/empleos-area-tecnol...,"Sorry, you have been blocked",<NA>,None,None,None


In [33]:
if df_jobs.empty:
    print("⛔ No hay datos que guardar.")
else:
    keep = ["Titulo del puesto","Descripcion","Distrito","Modo de trabajo","url"]
    for c in keep:
        if c not in df_jobs.columns: df_jobs[c] = None
    ts = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = os.path.join(OUT_DIR, f"bumeran_{ts}.csv")
    df_jobs[keep].to_csv(out_path, index=False, encoding="utf-8-sig", sep=";")
    print("✅ Guardado:", os.path.abspath(out_path))
    display(df_jobs[keep].head())


✅ Guardado: C:\Users\User\Documents\PROFE_CHRISTIAN\web_scrapping\output\bumeran_20250924_193906.csv


,Titulo del puesto,Descripcion,Distrito,Modo de trabajo,url
0,"Sorry, you have been blocked",<NA>,None,None,https://www.bumeran.com.pe/empleos-seniority-j...
1,"Sorry, you have been blocked",<NA>,None,None,https://www.bumeran.com.pe/empleos-seniority-g...
2,"Sorry, you have been blocked",<NA>,None,None,https://www.bumeran.com.pe/empleos.html
3,"Sorry, you have been blocked",<NA>,None,None,https://www.bumeran.com.pe/empleos-area-tecnol...
4,"Sorry, you have been blocked",<NA>,None,None,https://www.bumeran.com.pe/empleos-area-tecnol...


In [ ]:
## Extract Job Posting Links (Extraer enlaces de ofertas de empleo)

# --- Parte 2: parte_3_1_grupo7.ipynb ---

# 3.1 Extract Job Posting Links (Extraer enlaces de ofertas de empleo)

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

# Inicializar Chrome automáticamente con webdriver-manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

driver.get("https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html")

time.sleep(5)
print(driver.page_source[:2000])  # muestra los primeros 2000 caracteres del HTML

driver.quit()


<html lang="es-PE" data-react-helmet="lang"><head><script src="https://connect.facebook.net/signals/config/306895010943876?v=2.9.231&amp;r=stable&amp;domain=www.bumeran.com.pe&amp;hme=dcbbef44c997927c5af260dbc0059090127dce285cd316734e66d01ffe06084e&amp;ex_m=88%2C150%2C130%2C19%2C123%2C62%2C42%2C124%2C69%2C61%2C137%2C77%2C13%2C87%2C27%2C118%2C109%2C67%2C70%2C117%2C134%2C96%2C139%2C7%2C3%2C4%2C6%2C5%2C2%2C78%2C86%2C140%2C214%2C162%2C56%2C219%2C216%2C217%2C49%2C177%2C26%2C66%2C223%2C222%2C165%2C29%2C55%2C8%2C58%2C82%2C83%2C84%2C89%2C113%2C28%2C25%2C116%2C112%2C111%2C131%2C68%2C133%2C132%2C44%2C114%2C54%2C106%2C12%2C136%2C39%2C205%2C207%2C172%2C22%2C23%2C24%2C16%2C17%2C38%2C34%2C36%2C35%2C73%2C79%2C81%2C94%2C122%2C125%2C40%2C95%2C20%2C18%2C100%2C63%2C32%2C127%2C126%2C128%2C119%2C21%2C31%2C53%2C93%2C135%2C64%2C15%2C30%2C187%2C158%2C265%2C203%2C148%2C190%2C183%2C91%2C115%2C72%2C104%2C48%2C41%2C102%2C103%2C108%2C52%2C14%2C110%2C101%2C59%2C43%2C97%2C47%2C50%2C46%2C85%2C138%2C0%2C107%2C11%2C105

# De manera desagregada
Ingresamos a la página web de bumeran

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Esto descarga y configura automáticamente chromedriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

driver.get("https://www.bumeran.com.pe/")
driver.maximize_window()
time.sleep(5)
driver.quit()


# Ahora nos ubicamos en la pestaña "Buscar trabajo"

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.bumeran.com.pe/")
driver.maximize_window()

try:
    # Esperar hasta que el botón "Buscar trabajo" sea clickeable
    boton = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//button[normalize-space()='Buscar trabajo']"))
    )
    boton.click()
    print("✅ Botón 'Buscar trabajo' clickeado correctamente.")

except Exception as e:
    print("❌ No se pudo hacer clic en el botón:", e)

time.sleep(5)
driver.quit()


✅ Botón 'Buscar trabajo' clickeado correctamente.


# El filtro "Fecha de publicación"

In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.bumeran.com.pe/empleos-busqueda.html")
driver.maximize_window()

try:
    # Esperar el botón de filtro "Fecha de publicación"
    filtro_fecha = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//button//div[normalize-space()='Fecha de publicación']"))
    )
    filtro_fecha.click()
    print("✅ Filtro 'Fecha de publicación' abierto correctamente.")

except Exception as e:
    print("❌ No se pudo abrir el filtro de fecha:", e)

time.sleep(5)
driver.quit()


✅ Filtro 'Fecha de publicación' abierto correctamente.


# Aplicamos el filtro 'Menor a 15 días'

In [19]:
# 1️⃣ Importar librerías
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# 2️⃣ Crear driver con webdriver_manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.maximize_window()

# 3️⃣ Abrir Bumeran
driver.get("https://www.bumeran.com.pe/")

# 4️⃣ Esperar y cerrar ventana modal si existe
try:
    wait = WebDriverWait(driver, 10)
    cerrar_modal = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[3]/div/div/div[1]/div/div/div/button")))
    cerrar_modal.click()
    print("✅ Modal cerrado")
except:
    print("⚠️ No se encontró modal")

# 5️⃣ Click en 'Buscar trabajo'
try:
    buscar_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Buscar trabajo')]")))
    buscar_btn.click()
    print("✅ Botón 'Buscar trabajo' clickeado")
except:
    print("❌ No se pudo hacer clic en 'Buscar trabajo'")

# 6️⃣ Abrir filtro 'Fecha de publicación'
try:
    fecha_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Fecha de publicación')]")))
    fecha_btn.click()
    print("✅ Filtro 'Fecha de publicación' abierto")
except:
    print("❌ Error al abrir filtro 'Fecha de publicación'")

# 7️⃣ Seleccionar 'Menor a 15 días'
try:
    menor_15_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Menor a 15 días')]")))
    menor_15_btn.click()
    print("✅ Opción 'Menor a 15 días' seleccionada")
except:
    print("❌ Error al seleccionar 'Menor a 15 días'")

# 8️⃣ Esperar resultados (opcional)
time.sleep(5)

# 9️⃣ Cerrar navegador
driver.quit()


⚠️ No se encontró modal
✅ Botón 'Buscar trabajo' clickeado
✅ Filtro 'Fecha de publicación' abierto
✅ Opción 'Menor a 15 días' seleccionada


In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)


# Aplicamos los filtros 'Área' y 'Tecnología, Sistemas y Telecomunicaciones'

In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configurar Selenium
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 15)

try:
    # 1️⃣ Abrir Bumeran
    driver.get("https://www.bumeran.com.pe/")
    time.sleep(3)

    # 2️⃣ Cerrar modal si aparece
    try:
        modal_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Entendido')]")))
        modal_btn.click()
        print("✅ Modal cerrado")
    except:
        print("⚠️ No se encontró modal")

    # 3️⃣ Click en Buscar trabajo
    try:
        buscar_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Buscar trabajo')]")))
        buscar_btn.click()
        print("✅ Botón 'Buscar trabajo' clickeado")
    except Exception as e:
        print(f"❌ Error al hacer click en 'Buscar trabajo': {e}")

    # 4️⃣ Abrir filtro 'Fecha de publicación'
    try:
        fecha_filtro = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Fecha de publicación')]")))
        fecha_filtro.click()
        print("✅ Filtro 'Fecha de publicación' abierto")

        # Seleccionar "Menor a 15 días"
        fecha_15 = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Menor a 15 días')]")))
        fecha_15.click()
        print("✅ Opción 'Menor a 15 días' seleccionada")
    except Exception as e:
        print(f"❌ Error en filtro 'Fecha de publicación': {e}")

    # 5️⃣ Abrir filtro 'Área'
    try:
        area_filtro = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Área')]")))
        area_filtro.click()
        print("✅ Filtro 'Área' abierto")
    except Exception as e:
        print(f"❌ Error al abrir filtro 'Área': {e}")

    # 6️⃣ Seleccionar 'Tecnología, Sistemas y Telecomunicaciones'
    try:
        tecnologia_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Tecnología, Sistemas y Telecomunicaciones')]")
        ))
        driver.execute_script("arguments[0].scrollIntoView(true);", tecnologia_btn)
        driver.execute_script("arguments[0].click();", tecnologia_btn)
        print("✅ Área 'Tecnología, Sistemas y Telecomunicaciones' seleccionada")
    except Exception as e:
        print(f"❌ Error al seleccionar área 'Tecnología, Sistemas y Telecomunicaciones': {e}")

   
finally:
    print("\n🔚 Cerrando navegador...")
    time.sleep(10)
    driver.quit()


⚠️ No se encontró modal
✅ Botón 'Buscar trabajo' clickeado
✅ Filtro 'Fecha de publicación' abierto
✅ Opción 'Menor a 15 días' seleccionada
✅ Filtro 'Área' abierto
✅ Área 'Tecnología, Sistemas y Telecomunicaciones' seleccionada

🔚 Cerrando navegador...


# Aplicamos los filtros 'Subárea' y 'Programación'

In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configurar Selenium
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 15)

try:
    # 1️⃣ Abrir Bumeran
    driver.get("https://www.bumeran.com.pe/")
    time.sleep(3)

    # 2️⃣ Cerrar modal si aparece
    try:
        modal_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Entendido')]")))
        modal_btn.click()
        print("✅ Modal cerrado")
    except:
        print("⚠️ No se encontró modal")

    # 3️⃣ Click en Buscar trabajo
    try:
        buscar_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Buscar trabajo')]")))
        buscar_btn.click()
        print("✅ Botón 'Buscar trabajo' clickeado")
    except Exception as e:
        print(f"❌ Error al hacer click en 'Buscar trabajo': {e}")

    # 4️⃣ Abrir filtro 'Fecha de publicación'
    try:
        fecha_filtro = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Fecha de publicación')]")))
        fecha_filtro.click()
        print("✅ Filtro 'Fecha de publicación' abierto")

        # Seleccionar "Menor a 15 días"
        fecha_15 = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Menor a 15 días')]")))
        fecha_15.click()
        print("✅ Opción 'Menor a 15 días' seleccionada")
    except Exception as e:
        print(f"❌ Error en filtro 'Fecha de publicación': {e}")

    # 5️⃣ Abrir filtro 'Área'
    try:
        area_filtro = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Área')]")))
        area_filtro.click()
        print("✅ Filtro 'Área' abierto")
    except Exception as e:
        print(f"❌ Error al abrir filtro 'Área': {e}")

    # 6️⃣ Seleccionar 'Tecnología, Sistemas y Telecomunicaciones'
    try:
        tecnologia_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Tecnología, Sistemas y Telecomunicaciones')]")
        ))
        driver.execute_script("arguments[0].scrollIntoView(true);", tecnologia_btn)
        driver.execute_script("arguments[0].click();", tecnologia_btn)
        print("✅ Área 'Tecnología, Sistemas y Telecomunicaciones' seleccionada")
    except Exception as e:
        print(f"❌ Error al seleccionar área 'Tecnología, Sistemas y Telecomunicaciones': {e}")


    # 7️⃣ Abrir filtro 'Subárea'
    try:
        # Esperamos hasta que el botón de Subárea aparezca en el DOM
        subarea_filtro = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Subárea')]")
        ))
        subarea_filtro.click()
        print("✅ Filtro 'Subárea' abierto")
    except Exception as e:
        print(f"❌ Error al abrir filtro 'Subárea': {e}")

    # 8️⃣ Seleccionar 'Programación'
    try:
        programacion_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Programación')]")
        ))
        driver.execute_script("arguments[0].scrollIntoView(true);", programacion_btn)
        driver.execute_script("arguments[0].click();", programacion_btn)
        print("✅ Subárea 'Programación' seleccionada")
    except Exception as e:
        print(f"❌ Error al seleccionar subárea 'Programación': {e}")

finally:
    print("\n🔚 Cerrando navegador...")
    time.sleep(10)
    driver.quit()


⚠️ No se encontró modal
✅ Botón 'Buscar trabajo' clickeado
✅ Filtro 'Fecha de publicación' abierto
✅ Opción 'Menor a 15 días' seleccionada
✅ Filtro 'Área' abierto
✅ Área 'Tecnología, Sistemas y Telecomunicaciones' seleccionada
✅ Filtro 'Subárea' abierto
✅ Subárea 'Programación' seleccionada

🔚 Cerrando navegador...


# A continuación los filtros 'Departamento' y 'Lima'

In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configurar Selenium
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 15)

try:
    # 1️⃣ Abrir Bumeran
    driver.get("https://www.bumeran.com.pe/")
    time.sleep(3)

    # 2️⃣ Cerrar modal si aparece
    try:
        modal_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Entendido')]")))
        modal_btn.click()
        print("✅ Modal cerrado")
    except:
        print("⚠️ No se encontró modal")

    # 3️⃣ Click en Buscar trabajo
    try:
        buscar_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Buscar trabajo')]")))
        buscar_btn.click()
        print("✅ Botón 'Buscar trabajo' clickeado")
    except Exception as e:
        print(f"❌ Error al hacer click en 'Buscar trabajo': {e}")

    # 4️⃣ Abrir filtro 'Fecha de publicación'
    try:
        fecha_filtro = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Fecha de publicación')]")))
        fecha_filtro.click()
        print("✅ Filtro 'Fecha de publicación' abierto")

        # Seleccionar "Menor a 15 días"
        fecha_15 = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Menor a 15 días')]")))
        fecha_15.click()
        print("✅ Opción 'Menor a 15 días' seleccionada")
    except Exception as e:
        print(f"❌ Error en filtro 'Fecha de publicación': {e}")

    # 5️⃣ Abrir filtro 'Área'
    try:
        area_filtro = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Área')]")))
        area_filtro.click()
        print("✅ Filtro 'Área' abierto")
    except Exception as e:
        print(f"❌ Error al abrir filtro 'Área': {e}")

    # 6️⃣ Seleccionar 'Tecnología, Sistemas y Telecomunicaciones'
    try:
        tecnologia_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Tecnología, Sistemas y Telecomunicaciones')]")
        ))
        driver.execute_script("arguments[0].scrollIntoView(true);", tecnologia_btn)
        driver.execute_script("arguments[0].click();", tecnologia_btn)
        print("✅ Área 'Tecnología, Sistemas y Telecomunicaciones' seleccionada")
    except Exception as e:
        print(f"❌ Error al seleccionar área 'Tecnología, Sistemas y Telecomunicaciones': {e}")


    # 7️⃣ Abrir filtro 'Subárea'
    try:
        # Esperamos hasta que el botón de Subárea aparezca en el DOM
        subarea_filtro = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Subárea')]")
        ))
        subarea_filtro.click()
        print("✅ Filtro 'Subárea' abierto")
    except Exception as e:
        print(f"❌ Error al abrir filtro 'Subárea': {e}")

    # 8️⃣ Seleccionar 'Programación'
    try:
        programacion_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Programación')]")
        ))
        driver.execute_script("arguments[0].scrollIntoView(true);", programacion_btn)
        driver.execute_script("arguments[0].click();", programacion_btn)
        print("✅ Subárea 'Programación' seleccionada")
    except Exception as e:
        print(f"❌ Error al seleccionar subárea 'Programación': {e}")

    # 9️⃣ Abrir filtro 'Departamento'
    try:
        departamento_filtro = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Departamento')]")
        ))
        departamento_filtro.click()
        print("✅ Filtro 'Departamento' abierto")
    except Exception as e:
        print(f"❌ Error al abrir filtro 'Departamento': {e}")

    # 🔟 Seleccionar 'Lima' (puedes cambiar por otro departamento)
    try:
        lima_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Lima')]")
        ))
        driver.execute_script("arguments[0].scrollIntoView(true);", lima_btn)
        driver.execute_script("arguments[0].click();", lima_btn)
        print("✅ Departamento 'Lima' seleccionado")
    except Exception as e:
        print(f"❌ Error al seleccionar departamento 'Lima': {e}")

finally:
    print("\n🔚 Cerrando navegador...")
    time.sleep(10)
    driver.quit()


⚠️ No se encontró modal
✅ Botón 'Buscar trabajo' clickeado
✅ Filtro 'Fecha de publicación' abierto
✅ Opción 'Menor a 15 días' seleccionada
✅ Filtro 'Área' abierto
✅ Área 'Tecnología, Sistemas y Telecomunicaciones' seleccionada
✅ Filtro 'Subárea' abierto
✅ Subárea 'Programación' seleccionada
✅ Filtro 'Departamento' abierto
✅ Departamento 'Lima' seleccionado

🔚 Cerrando navegador...


# Aplicamos los filtros "Carga horaria" y "Full time"

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configurar Selenium
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 15)

try:
    # 1️⃣ Abrir Bumeran
    driver.get("https://www.bumeran.com.pe/")
    time.sleep(3)

    # 2️⃣ Cerrar modal si aparece
    try:
        modal_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Entendido')]")))
        modal_btn.click()
        print("✅ Modal cerrado")
    except:
        print("⚠️ No se encontró modal")

    # 3️⃣ Click en Buscar trabajo
    try:
        buscar_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Buscar trabajo')]")))
        buscar_btn.click()
        print("✅ Botón 'Buscar trabajo' clickeado")
    except Exception as e:
        print(f"❌ Error al hacer click en 'Buscar trabajo': {e}")

    # 4️⃣ Abrir filtro 'Fecha de publicación'
    try:
        fecha_filtro = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Fecha de publicación')]")))
        fecha_filtro.click()
        print("✅ Filtro 'Fecha de publicación' abierto")

        # Seleccionar "Menor a 15 días"
        fecha_15 = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Menor a 15 días')]")))
        fecha_15.click()
        print("✅ Opción 'Menor a 15 días' seleccionada")
    except Exception as e:
        print(f"❌ Error en filtro 'Fecha de publicación': {e}")

    # 5️⃣ Abrir filtro 'Área'
    try:
        area_filtro = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Área')]")))
        area_filtro.click()
        print("✅ Filtro 'Área' abierto")
    except Exception as e:
        print(f"❌ Error al abrir filtro 'Área': {e}")

    # 6️⃣ Seleccionar 'Tecnología, Sistemas y Telecomunicaciones'
    try:
        tecnologia_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Tecnología, Sistemas y Telecomunicaciones')]")
        ))
        driver.execute_script("arguments[0].scrollIntoView(true);", tecnologia_btn)
        driver.execute_script("arguments[0].click();", tecnologia_btn)
        print("✅ Área 'Tecnología, Sistemas y Telecomunicaciones' seleccionada")
    except Exception as e:
        print(f"❌ Error al seleccionar área 'Tecnología, Sistemas y Telecomunicaciones': {e}")


    # 7️⃣ Abrir filtro 'Subárea'
    try:
        # Esperamos hasta que el botón de Subárea aparezca en el DOM
        subarea_filtro = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Subárea')]")
        ))
        subarea_filtro.click()
        print("✅ Filtro 'Subárea' abierto")
    except Exception as e:
        print(f"❌ Error al abrir filtro 'Subárea': {e}")

    # 8️⃣ Seleccionar 'Programación'
    try:
        programacion_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Programación')]")
        ))
        driver.execute_script("arguments[0].scrollIntoView(true);", programacion_btn)
        driver.execute_script("arguments[0].click();", programacion_btn)
        print("✅ Subárea 'Programación' seleccionada")
    except Exception as e:
        print(f"❌ Error al seleccionar subárea 'Programación': {e}")

    # 9️⃣ Abrir filtro 'Departamento'
    try:
        departamento_filtro = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Departamento')]")
        ))
        departamento_filtro.click()
        print("✅ Filtro 'Departamento' abierto")
    except Exception as e:
        print(f"❌ Error al abrir filtro 'Departamento': {e}")

    # 🔟 Seleccionar 'Lima' (puedes cambiar por otro departamento)
    try:
        lima_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Lima')]")
        ))
        driver.execute_script("arguments[0].scrollIntoView(true);", lima_btn)
        driver.execute_script("arguments[0].click();", lima_btn)
        print("✅ Departamento 'Lima' seleccionado")
    except Exception as e:
        print(f"❌ Error al seleccionar departamento 'Lima': {e}")


    # 1️⃣1️⃣ Abrir filtro 'Carga horaria'
    try:
        carga_horaria_filtro = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Carga horaria')]")
        ))
        carga_horaria_filtro.click()
        print("✅ Filtro 'Carga horaria' abierto")
    except Exception as e:
        print(f"❌ Error al abrir filtro 'Carga horaria': {e}")

    # 1️⃣2️⃣ Seleccionar 'Full-time'
    try:
        fulltime_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Full-time') or contains(.,'Tiempo completo')]")
        ))
        driver.execute_script("arguments[0].scrollIntoView(true);", fulltime_btn)
        driver.execute_script("arguments[0].click();", fulltime_btn)
        print("✅ Carga horaria 'Full-time' seleccionada")
    except Exception as e:
        print(f"❌ Error al seleccionar carga horaria 'Full-time': {e}")

finally:
    print("\n🔚 Cerrando navegador...")
    time.sleep(10)
    driver.quit()


# Finalmente, se da a conocer las primeras 20 ofertas de empleo

In [55]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configurar Selenium
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 15)

try:
    # 1️⃣ Abrir Bumeran
    driver.get("https://www.bumeran.com.pe/")
    time.sleep(3)

    # 2️⃣ Cerrar modal si aparece
    try:
        modal_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Entendido')]")))
        modal_btn.click()
        print("✅ Modal cerrado")
    except:
        print("⚠️ No se encontró modal")

    # 3️⃣ Click en Buscar trabajo
    try:
        buscar_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Buscar trabajo')]")))
        buscar_btn.click()
        print("✅ Botón 'Buscar trabajo' clickeado")
    except Exception as e:
        print(f"❌ Error al hacer click en 'Buscar trabajo': {e}")

    # 4️⃣ Abrir filtro 'Fecha de publicación'
    try:
        fecha_filtro = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Fecha de publicación')]")))
        fecha_filtro.click()
        print("✅ Filtro 'Fecha de publicación' abierto")

        # Seleccionar "Menor a 15 días"
        fecha_15 = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Menor a 15 días')]")))
        fecha_15.click()
        print("✅ Opción 'Menor a 15 días' seleccionada")
    except Exception as e:
        print(f"❌ Error en filtro 'Fecha de publicación': {e}")

    # 5️⃣ Abrir filtro 'Área'
    try:
        area_filtro = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(.,'Área')]")))
        area_filtro.click()
        print("✅ Filtro 'Área' abierto")
    except Exception as e:
        print(f"❌ Error al abrir filtro 'Área': {e}")

    # 6️⃣ Seleccionar 'Tecnología, Sistemas y Telecomunicaciones'
    try:
        tecnologia_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Tecnología, Sistemas y Telecomunicaciones')]")
        ))
        driver.execute_script("arguments[0].scrollIntoView(true);", tecnologia_btn)
        driver.execute_script("arguments[0].click();", tecnologia_btn)
        print("✅ Área 'Tecnología, Sistemas y Telecomunicaciones' seleccionada")
    except Exception as e:
        print(f"❌ Error al seleccionar área 'Tecnología, Sistemas y Telecomunicaciones': {e}")


    # 7️⃣ Abrir filtro 'Subárea'
    try:
        # Esperamos hasta que el botón de Subárea aparezca en el DOM
        subarea_filtro = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Subárea')]")
        ))
        subarea_filtro.click()
        print("✅ Filtro 'Subárea' abierto")
    except Exception as e:
        print(f"❌ Error al abrir filtro 'Subárea': {e}")

    # 8️⃣ Seleccionar 'Programación'
    try:
        programacion_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Programación')]")
        ))
        driver.execute_script("arguments[0].scrollIntoView(true);", programacion_btn)
        driver.execute_script("arguments[0].click();", programacion_btn)
        print("✅ Subárea 'Programación' seleccionada")
    except Exception as e:
        print(f"❌ Error al seleccionar subárea 'Programación': {e}")

    # 9️⃣ Abrir filtro 'Departamento'
    try:
        departamento_filtro = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Departamento')]")
        ))
        departamento_filtro.click()
        print("✅ Filtro 'Departamento' abierto")
    except Exception as e:
        print(f"❌ Error al abrir filtro 'Departamento': {e}")

    # 🔟 Seleccionar 'Lima' (puedes cambiar por otro departamento)
    try:
        lima_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Lima')]")
        ))
        driver.execute_script("arguments[0].scrollIntoView(true);", lima_btn)
        driver.execute_script("arguments[0].click();", lima_btn)
        print("✅ Departamento 'Lima' seleccionado")
    except Exception as e:
        print(f"❌ Error al seleccionar departamento 'Lima': {e}")


    # 1️⃣1️⃣ Abrir filtro 'Carga horaria'
    try:
        carga_horaria_filtro = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Carga horaria')]")
        ))
        carga_horaria_filtro.click()
        print("✅ Filtro 'Carga horaria' abierto")
    except Exception as e:
        print(f"❌ Error al abrir filtro 'Carga horaria': {e}")

    # 1️⃣2️⃣ Seleccionar 'Full-time'
    try:
        fulltime_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(.,'Full-time') or contains(.,'Tiempo completo')]")
        ))
        driver.execute_script("arguments[0].scrollIntoView(true);", fulltime_btn)
        driver.execute_script("arguments[0].click();", fulltime_btn)
        print("✅ Carga horaria 'Full-time' seleccionada")
    except Exception as e:
        print(f"❌ Error al seleccionar carga horaria 'Full-time': {e}")

    # 1️⃣3️⃣ Extraer enlaces de ofertas de empleo
    try:
        time.sleep(5)  # Esperar a que carguen los resultados después de aplicar filtros
        ofertas = wait.until(EC.presence_of_all_elements_located(
            (By.XPATH, "//a[contains(@href,'/empleos/')]")
        ))

        enlaces = [oferta.get_attribute("href") for oferta in ofertas]

        print(f"📊 Se encontraron {len(enlaces)} ofertas de empleo")
        for i, enlace in enumerate(enlaces[:30], start=1):  # Mostrar solo las primeras 10
            print(f"{i}. {enlace}")

    except Exception as e:
        print(f"❌ Error al extraer enlaces de ofertas: {e}")

finally:
    print("\n🔚 Cerrando navegador...")
    time.sleep(10)
    driver.quit()


⚠️ No se encontró modal
✅ Botón 'Buscar trabajo' clickeado
✅ Filtro 'Fecha de publicación' abierto
✅ Opción 'Menor a 15 días' seleccionada
✅ Filtro 'Área' abierto
✅ Área 'Tecnología, Sistemas y Telecomunicaciones' seleccionada
✅ Filtro 'Subárea' abierto
✅ Subárea 'Programación' seleccionada
✅ Filtro 'Departamento' abierto
✅ Departamento 'Lima' seleccionado
✅ Filtro 'Carga horaria' abierto
✅ Carga horaria 'Full-time' seleccionada
📊 Se encontraron 20 ofertas de empleo
1. https://www.bumeran.com.pe/empleos/software-engineer-senior-grupo-gloria-1117976663.html
2. https://www.bumeran.com.pe/empleos/programador-frontend-senior-sonda-del-peru-s.a.-1117973000.html
3. https://www.bumeran.com.pe/empleos/analista-programador-java-1117972678.html
4. https://www.bumeran.com.pe/empleos/analista-programador-sede-ate-bumeran-selecta-1117969140.html
5. https://www.bumeran.com.pe/empleos/trainee-programador-jr-building-software-1117968407.html
6. https://www.bumeran.com.pe/empleos/desarrollador-full-sta

# --- Parte 3: Parte_3_2_Grupo_7_Contreras.ipynb ---

In [21]:
# --- LIBRERÍAS NECESARIAS ---
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

# --- CONFIGURACIÓN DE SELENIUM ---
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
# options.add_argument("--headless") # Descomenta para que el navegador corra en segundo plano
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 15)
enlaces = []
datos_trabajos = []


try:
    # --- ETAPA 1: OBTENER LOS ENLACES ---
    print("🚀 INICIANDO ETAPA 1: Obteniendo los enlaces de las ofertas...")
    url_con_filtros = "https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html"
    driver.get(url_con_filtros)
    time.sleep(5)
    print("✅ Página de resultados con filtros cargada.")

    ofertas_raw = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "a")))
    enlaces_temp = [oferta.get_attribute("href") for oferta in ofertas_raw if oferta.get_attribute("href")]
    enlaces = list(set([link for link in enlaces_temp if link and '/empleos/' in link and not '/postulante/' in link]))
    print(f"\n👍 ETAPA 1 COMPLETADA: Se encontraron {len(enlaces)} ofertas de trabajo únicas.")

    # --- ETAPA 2: LÓGICA DE EXTRACCIÓN MULTI-INTENTO ---
    print("\n🚀 INICIANDO ETAPA 2: Extrayendo detalles de cada oferta...")
    for i, link in enumerate(enlaces, start=1):
        try:
            driver.get(link)
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "h1")))
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # --- ESTRATEGIA DE EXTRACCIÓN ROBUSTA ---
            
            # 1. TÍTULO: Siempre es el <h1>.
            titulo = soup.find('h1').get_text(strip=True) if soup.find('h1') else "No encontrado"
            
            # 2. DESCRIPCIÓN: Se intentan varias formas.
            descripcion = "No encontrada"
            desc_header = soup.find('h3', string='Descripción del puesto')
            if desc_header:
                desc_p = desc_header.find_parent('div').find_next_sibling('p')
                if desc_p: descripcion = desc_p.get_text(strip=True, separator='\\n')
            if descripcion == "No encontrada":
                desc_div = soup.find('p', class_='sc-hqCxVl')
                if desc_div: descripcion = desc_div.get_text(strip=True, separator='\\n')

            # 3. DISTRITO: Se intentan varias formas.
            distrito = "No encontrado"
            distrito_tag = soup.find('h2', class_='sc-gbKBEV')
            if distrito_tag: distrito = distrito_tag.get_text(strip=True)
            if distrito == "No encontrado":
                all_h2 = soup.find_all('h2')
                for h2 in all_h2:
                    if "Peru" in h2.get_text():
                        distrito = h2.get_text(strip=True)
                        break
            
            # 4. MODALIDAD: Búsqueda global de palabras clave.
            modalidad = "No especificada"
            page_text = soup.body.get_text()
            if "Híbrido" in page_text: modalidad = "Híbrido"
            elif "Presencial" in page_text: modalidad = "Presencial"
            elif "Home Office" in page_text or "Remoto" in page_text: modalidad = "Remoto"

            datos_trabajos.append({
                "Job Title": titulo,
                "Description": descripcion,
                "District": distrito,
                "Work Mode": modalidad
            })
            print(f"  -> ✅ Extraída oferta {i}/{len(enlaces)}: {titulo}")
        except Exception as e:
            print(f"  -> ❌ Error en la oferta {i} ({link}) - {type(e).__name__}: {e}")
            
    print("\n👍 ETAPA 2 COMPLETADA: Se extrajeron los detalles.")

finally:
    print("\n🔚 Cerrando navegador...")
    driver.quit()

# --- ETAPA 3: GUARDAR LOS DATOS EN CSV ---
if datos_trabajos:
    print("\n🚀 INICIANDO ETAPA 3: Guardando datos en archivo CSV...")
    df = pd.DataFrame(datos_trabajos)
    output_dir = 'output'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    ruta_salida = os.path.join(output_dir, 'bumeran_jobs.csv')
    df.to_csv(ruta_salida, index=False, encoding='utf-8')
    print(f"\n🎉 ¡TAREA COMPLETADA! Archivo guardado en: {ruta_salida}")
    display(df)
else:
    print("\n⚠️ No se encontraron datos para guardar en el archivo CSV.")


    

🚀 INICIANDO ETAPA 1: Obteniendo los enlaces de las ofertas...
✅ Página de resultados con filtros cargada.

👍 ETAPA 1 COMPLETADA: Se encontraron 20 ofertas de trabajo únicas.

🚀 INICIANDO ETAPA 2: Extrayendo detalles de cada oferta...
  -> ✅ Extraída oferta 1/20: Analista Programador Postgre/Oracle/Híbrido
  -> ✅ Extraída oferta 2/20: Analista Programador - Sede ATE
  -> ✅ Extraída oferta 3/20: Desarrollador Full Stack / Surco / Presencial
  -> ✅ Extraída oferta 4/20: Analista Programador Java
  -> ✅ Extraída oferta 5/20: Desarrollador Backend Semi-Senior
  -> ✅ Extraída oferta 6/20: Desarrollador/ Líder Técnico
  -> ✅ Extraída oferta 7/20: Desarrollador Full Stack Node.js, C#, Java, AngularJS
  -> ✅ Extraída oferta 8/20: Analista de Imagenes Hospitalarias / Sistemas
  -> ✅ Extraída oferta 9/20: Senior React Developer
  -> ✅ Extraída oferta 10/20: Java Developer - Senior
  -> ✅ Extraída oferta 11/20: Trainee Programador Jr
  -> ✅ Extraída oferta 12/20: Programador Full Stack (Java, Reac

,Job Title,Description,District,Work Mode
0,Analista Programador Postgre/Oracle/Híbrido,"Por encargo de nuestro cliente, importante emp...","San Borja, Lima, Peru",Híbrido
1,Analista Programador - Sede ATE,¡Buscamos a nuestro próximo/a Analista Program...,"Ate, Lima, Peru",Presencial
2,Desarrollador Full Stack / Surco / Presencial,"En\nWell Human Resources\n, estamos en la búsq...","Santiago de Surco, Lima, Peru",Híbrido
3,Analista Programador Java,Analista Programador Java\nResumen del Puesto\...,"Lima, Lima, Peru",Híbrido
4,Desarrollador Backend Semi-Senior,Métrica Perú es la primera filial latinoameric...,"Lima, Lima, Peru",Híbrido
5,Desarrollador/ Líder Técnico,En Protiviti Perú buscamos un\nDesarrollador d...,"La Victoria, Lima, Peru",Híbrido
6,"Desarrollador Full Stack Node.js, C#, Java, An...",Puesto: Desarrollador Full Stack Senior\n📍 Mod...,"Lima, Lima, Peru",Híbrido
7,Analista de Imagenes Hospitalarias / Sistemas,Grupo San Pablo te invita a unirte a su equipo...,"Santiago de Surco, Lima, Peru",Híbrido
8,Senior React Developer,Información Importante\nUbicación del cliente:...,Publicado ayer,Híbrido
9,Java Developer - Senior,Actualmente nos encontramos en búsqueda de 02 ...,"Chorrillos, Lima, Peru",Híbrido
